# Chapter 9: LangGraph Deployment 실습

이 노트북은 LangGraph 애플리케이션의 배포와 프로덕션 환경 구성을 실습합니다.

## 환경 설정

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = input("OpenAI API Key를 입력하세요: ")

## 1. LangGraph Configuration

In [ ]:
import json
from typing import Dict, Any, TypedDict, Optional
from dataclasses import dataclass
from pydantic import BaseModel, Field

# LangGraph 설정 클래스
@dataclass
class LangGraphConfig:
    """LangGraph 애플리케이션 설정"""
    app_name: str
    version: str
    environment: str  # development, staging, production
    api_keys: Dict[str, str]
    database_url: Optional[str] = None
    redis_url: Optional[str] = None
    max_workers: int = 4
    timeout: int = 30
    retry_policy: Dict[str, Any] = None
    
    def to_dict(self) -> Dict:
        return {
            "app_name": self.app_name,
            "version": self.version,
            "environment": self.environment,
            "max_workers": self.max_workers,
            "timeout": self.timeout,
            "retry_policy": self.retry_policy or {
                "max_retries": 3,
                "backoff_factor": 2,
                "max_backoff": 60
            }
        }
    
    def save_to_file(self, filepath: str):
        """설정을 파일로 저장"""
        config_dict = self.to_dict()
        with open(filepath, 'w') as f:
            json.dump(config_dict, f, indent=2)
        print(f"설정이 {filepath}에 저장되었습니다.")
    
    @classmethod
    def load_from_file(cls, filepath: str):
        """파일에서 설정 로드"""
        with open(filepath, 'r') as f:
            config_dict = json.load(f)
        return cls(**config_dict)

# 환경별 설정 생성
def create_environment_config(env: str) -> LangGraphConfig:
    """환경별 설정 생성"""
    base_config = {
        "app_name": "langgraph-app",
        "version": "1.0.0",
        "api_keys": {
            "openai": os.getenv("OPENAI_API_KEY", ""),
            "anthropic": os.getenv("ANTHROPIC_API_KEY", "")
        }
    }
    
    if env == "development":
        return LangGraphConfig(
            **base_config,
            environment="development",
            database_url="sqlite:///dev.db",
            max_workers=2,
            timeout=60
        )
    elif env == "staging":
        return LangGraphConfig(
            **base_config,
            environment="staging",
            database_url=os.getenv("DATABASE_URL"),
            redis_url=os.getenv("REDIS_URL"),
            max_workers=4,
            timeout=30
        )
    elif env == "production":
        return LangGraphConfig(
            **base_config,
            environment="production",
            database_url=os.getenv("DATABASE_URL"),
            redis_url=os.getenv("REDIS_URL"),
            max_workers=8,
            timeout=30,
            retry_policy={
                "max_retries": 5,
                "backoff_factor": 2,
                "max_backoff": 120
            }
        )
    
    return LangGraphConfig(**base_config, environment="development")

# 테스트
dev_config = create_environment_config("development")
prod_config = create_environment_config("production")

print("개발 환경 설정:")
print(json.dumps(dev_config.to_dict(), indent=2))

print("\n프로덕션 환경 설정:")
print(json.dumps(prod_config.to_dict(), indent=2))

## 2. Graph Deployment Service

In [ ]:
from typing import List, Callable
from langgraph.graph import StateGraph, END, MessagesState
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
import asyncio
import time
from datetime import datetime

# 배포 가능한 그래프 서비스
class GraphDeploymentService:
    """LangGraph 배포 서비스"""
    
    def __init__(self, config: LangGraphConfig):
        self.config = config
        self.graphs = {}
        self.metrics = {
            "requests": 0,
            "errors": 0,
            "avg_latency": 0,
            "start_time": datetime.now()
        }
    
    def register_graph(self, name: str, graph_builder: Callable):
        """그래프 등록"""
        try:
            graph = graph_builder()
            self.graphs[name] = graph
            print(f"✅ 그래프 '{name}' 등록 완료")
        except Exception as e:
            print(f"❌ 그래프 '{name}' 등록 실패: {e}")
    
    async def invoke_graph(self, graph_name: str, input_data: Dict) -> Dict:
        """그래프 실행"""
        if graph_name not in self.graphs:
            raise ValueError(f"그래프 '{graph_name}'을 찾을 수 없습니다.")
        
        start_time = time.time()
        self.metrics["requests"] += 1
        
        try:
            # 타임아웃 적용
            result = await asyncio.wait_for(
                self._execute_graph(graph_name, input_data),
                timeout=self.config.timeout
            )
            
            # 메트릭 업데이트
            latency = time.time() - start_time
            self._update_metrics(latency)
            
            return result
        except asyncio.TimeoutError:
            self.metrics["errors"] += 1
            raise TimeoutError(f"그래프 실행이 {self.config.timeout}초를 초과했습니다.")
        except Exception as e:
            self.metrics["errors"] += 1
            raise e
    
    async def _execute_graph(self, graph_name: str, input_data: Dict) -> Dict:
        """실제 그래프 실행"""
        graph = self.graphs[graph_name]
        # 동기 함수를 비동기로 실행
        loop = asyncio.get_event_loop()
        result = await loop.run_in_executor(None, graph.invoke, input_data)
        return result
    
    def _update_metrics(self, latency: float):
        """메트릭 업데이트"""
        current_avg = self.metrics["avg_latency"]
        total_requests = self.metrics["requests"]
        self.metrics["avg_latency"] = (
            (current_avg * (total_requests - 1) + latency) / total_requests
        )
    
    def get_health_status(self) -> Dict:
        """헬스 체크"""
        uptime = (datetime.now() - self.metrics["start_time"]).total_seconds()
        error_rate = (
            self.metrics["errors"] / self.metrics["requests"]
            if self.metrics["requests"] > 0 else 0
        )
        
        return {
            "status": "healthy" if error_rate < 0.1 else "degraded",
            "uptime_seconds": uptime,
            "total_requests": self.metrics["requests"],
            "error_rate": error_rate,
            "avg_latency_ms": self.metrics["avg_latency"] * 1000,
            "registered_graphs": list(self.graphs.keys())
        }

# 샘플 그래프 빌더
def build_chat_graph():
    """채팅 그래프 생성"""
    workflow = StateGraph(MessagesState)
    
    def chat_node(state: MessagesState):
        llm = ChatOpenAI(model="gpt-4o-mini")
        response = llm.invoke(state["messages"])
        return {"messages": [response]}
    
    workflow.add_node("chat", chat_node)
    workflow.set_entry_point("chat")
    workflow.add_edge("chat", END)
    
    return workflow.compile()

def build_analysis_graph():
    """분석 그래프 생성"""
    class AnalysisState(TypedDict):
        text: str
        analysis: str
    
    workflow = StateGraph(AnalysisState)
    
    def analyze_node(state: AnalysisState):
        llm = ChatOpenAI(model="gpt-4o-mini")
        prompt = f"다음 텍스트를 분석하세요: {state['text']}"
        response = llm.invoke(prompt)
        return {"analysis": response.content}
    
    workflow.add_node("analyze", analyze_node)
    workflow.set_entry_point("analyze")
    workflow.add_edge("analyze", END)
    
    return workflow.compile()

# 서비스 배포 시뮬레이션
async def deploy_and_test():
    """배포 및 테스트"""
    # 설정 생성
    config = create_environment_config("staging")
    
    # 서비스 초기화
    service = GraphDeploymentService(config)
    
    # 그래프 등록
    service.register_graph("chat", build_chat_graph)
    service.register_graph("analysis", build_analysis_graph)
    
    print("\n📊 서비스 상태:")
    print(json.dumps(service.get_health_status(), indent=2))
    
    # 테스트 요청
    print("\n🚀 테스트 요청 실행:")
    
    # 채팅 그래프 테스트
    chat_result = await service.invoke_graph(
        "chat",
        {"messages": [HumanMessage(content="안녕하세요!")]}
    )
    print(f"채팅 응답: {chat_result['messages'][-1].content[:50]}...")
    
    # 분석 그래프 테스트
    analysis_result = await service.invoke_graph(
        "analysis",
        {"text": "LangGraph는 강력한 프레임워크입니다."}
    )
    print(f"분석 결과: {analysis_result['analysis'][:50]}...")
    
    # 최종 메트릭
    print("\n📈 최종 메트릭:")
    print(json.dumps(service.get_health_status(), indent=2))

# 실행
await deploy_and_test()

## 3. API Server Implementation

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import threading
import uuid
from queue import Queue
from typing import Any

# API 서버 구현
class LangGraphAPIServer:
    """LangGraph API 서버"""
    
    def __init__(self, service: GraphDeploymentService, port: int = 5000):
        self.service = service
        self.port = port
        self.app = Flask(__name__)
        CORS(self.app)
        self.request_queue = Queue()
        self.response_cache = {}
        self._setup_routes()
    
    def _setup_routes(self):
        """API 라우트 설정"""
        
        @self.app.route('/health', methods=['GET'])
        def health_check():
            return jsonify(self.service.get_health_status())
        
        @self.app.route('/graphs', methods=['GET'])
        def list_graphs():
            return jsonify({
                "graphs": list(self.service.graphs.keys()),
                "count": len(self.service.graphs)
            })
        
        @self.app.route('/invoke/<graph_name>', methods=['POST'])
        def invoke_graph(graph_name: str):
            try:
                data = request.json
                request_id = str(uuid.uuid4())
                
                # 요청을 큐에 추가
                self.request_queue.put({
                    "id": request_id,
                    "graph": graph_name,
                    "data": data
                })
                
                # 동기 실행 (실제로는 비동기 처리 권장)
                result = self._process_request(graph_name, data)
                
                return jsonify({
                    "request_id": request_id,
                    "status": "success",
                    "result": result
                })
            except Exception as e:
                return jsonify({
                    "status": "error",
                    "error": str(e)
                }), 500
        
        @self.app.route('/batch', methods=['POST'])
        def batch_invoke():
            """배치 처리"""
            try:
                batch_requests = request.json.get("requests", [])
                results = []
                
                for req in batch_requests:
                    graph_name = req.get("graph")
                    data = req.get("data")
                    
                    try:
                        result = self._process_request(graph_name, data)
                        results.append({
                            "status": "success",
                            "result": result
                        })
                    except Exception as e:
                        results.append({
                            "status": "error",
                            "error": str(e)
                        })
                
                return jsonify({"results": results})
            except Exception as e:
                return jsonify({"error": str(e)}), 500
    
    def _process_request(self, graph_name: str, data: Dict) -> Any:
        """요청 처리"""
        # 실제로는 비동기 처리가 필요
        import asyncio
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        result = loop.run_until_complete(
            self.service.invoke_graph(graph_name, data)
        )
        loop.close()
        return result
    
    def run(self, debug: bool = False):
        """서버 실행"""
        print(f"🚀 API 서버가 포트 {self.port}에서 시작됩니다...")
        self.app.run(port=self.port, debug=debug)

# API 클라이언트 예제
class LangGraphClient:
    """LangGraph API 클라이언트"""
    
    def __init__(self, base_url: str):
        self.base_url = base_url
    
    def health_check(self) -> Dict:
        import requests
        response = requests.get(f"{self.base_url}/health")
        return response.json()
    
    def list_graphs(self) -> List[str]:
        import requests
        response = requests.get(f"{self.base_url}/graphs")
        return response.json()["graphs"]
    
    def invoke(self, graph_name: str, data: Dict) -> Dict:
        import requests
        response = requests.post(
            f"{self.base_url}/invoke/{graph_name}",
            json=data
        )
        return response.json()
    
    def batch_invoke(self, requests: List[Dict]) -> List[Dict]:
        import requests
        response = requests.post(
            f"{self.base_url}/batch",
            json={"requests": requests}
        )
        return response.json()["results"]

# API 서버 시뮬레이션
def simulate_api_server():
    """API 서버 시뮬레이션"""
    print("API 서버 시뮬레이션")
    print("=" * 60)
    
    # 설정 및 서비스 생성
    config = create_environment_config("development")
    service = GraphDeploymentService(config)
    
    # 그래프 등록
    service.register_graph("chat", build_chat_graph)
    service.register_graph("analysis", build_analysis_graph)
    
    # API 서버 생성 (실제 실행하지 않음)
    api_server = LangGraphAPIServer(service, port=5000)
    
    print("\n📋 API 엔드포인트:")
    print("- GET  /health       : 헬스 체크")
    print("- GET  /graphs       : 그래프 목록")
    print("- POST /invoke/<name>: 그래프 실행")
    print("- POST /batch        : 배치 처리")
    
    print("\n💡 서버 실행 명령:")
    print("api_server.run(debug=True)")
    
    # 클라이언트 사용 예제
    print("\n📱 클라이언트 사용 예제:")
    print("""
client = LangGraphClient("http://localhost:5000")

# 헬스 체크
health = client.health_check()
print(health)

# 그래프 실행
result = client.invoke("chat", {
    "messages": [{"role": "user", "content": "Hello!"}]
})
print(result)
    """)

simulate_api_server()

## 4. Docker Deployment

In [ ]:
# Docker 배포 설정 생성
def create_dockerfile() -> str:
    """Dockerfile 생성"""
    dockerfile_content = """
# Base image
FROM python:3.11-slim

# Set working directory
WORKDIR /app

# Install system dependencies
RUN apt-get update && apt-get install -y \\
    gcc \\
    g++ \\
    && rm -rf /var/lib/apt/lists/*

# Copy requirements
COPY requirements.txt .

# Install Python dependencies
RUN pip install --no-cache-dir -r requirements.txt

# Copy application code
COPY . .

# Set environment variables
ENV PYTHONUNBUFFERED=1
ENV LANGGRAPH_ENV=production

# Expose port
EXPOSE 8000

# Health check
HEALTHCHECK --interval=30s --timeout=30s --start-period=5s --retries=3 \\
    CMD python -c "import requests; requests.get('http://localhost:8000/health')" || exit 1

# Run application
CMD ["python", "-m", "uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]
"""
    return dockerfile_content.strip()

def create_docker_compose() -> str:
    """Docker Compose 파일 생성"""
    docker_compose_content = """
version: '3.8'

services:
  langgraph-app:
    build: .
    container_name: langgraph-service
    ports:
      - "8000:8000"
    environment:
      - OPENAI_API_KEY=${OPENAI_API_KEY}
      - DATABASE_URL=postgresql://user:password@db:5432/langgraph
      - REDIS_URL=redis://redis:6379
      - LANGGRAPH_ENV=production
    depends_on:
      - db
      - redis
    restart: unless-stopped
    networks:
      - langgraph-network
    volumes:
      - ./logs:/app/logs
    
  db:
    image: postgres:15
    container_name: langgraph-db
    environment:
      - POSTGRES_USER=user
      - POSTGRES_PASSWORD=password
      - POSTGRES_DB=langgraph
    volumes:
      - postgres_data:/var/lib/postgresql/data
    networks:
      - langgraph-network
    
  redis:
    image: redis:7-alpine
    container_name: langgraph-redis
    networks:
      - langgraph-network
    volumes:
      - redis_data:/data
  
  nginx:
    image: nginx:alpine
    container_name: langgraph-nginx
    ports:
      - "80:80"
      - "443:443"
    volumes:
      - ./nginx.conf:/etc/nginx/nginx.conf
      - ./ssl:/etc/nginx/ssl
    depends_on:
      - langgraph-app
    networks:
      - langgraph-network

networks:
  langgraph-network:
    driver: bridge

volumes:
  postgres_data:
  redis_data:
"""
    return docker_compose_content.strip()

def create_kubernetes_deployment() -> str:
    """Kubernetes 배포 YAML 생성"""
    k8s_deployment = """
apiVersion: apps/v1
kind: Deployment
metadata:
  name: langgraph-deployment
  labels:
    app: langgraph
spec:
  replicas: 3
  selector:
    matchLabels:
      app: langgraph
  template:
    metadata:
      labels:
        app: langgraph
    spec:
      containers:
      - name: langgraph
        image: langgraph-app:latest
        ports:
        - containerPort: 8000
        env:
        - name: OPENAI_API_KEY
          valueFrom:
            secretKeyRef:
              name: langgraph-secrets
              key: openai-api-key
        - name: DATABASE_URL
          valueFrom:
            secretKeyRef:
              name: langgraph-secrets
              key: database-url
        resources:
          requests:
            memory: "256Mi"
            cpu: "250m"
          limits:
            memory: "512Mi"
            cpu: "500m"
        livenessProbe:
          httpGet:
            path: /health
            port: 8000
          initialDelaySeconds: 30
          periodSeconds: 10
        readinessProbe:
          httpGet:
            path: /health
            port: 8000
          initialDelaySeconds: 5
          periodSeconds: 5
---
apiVersion: v1
kind: Service
metadata:
  name: langgraph-service
spec:
  selector:
    app: langgraph
  ports:
    - protocol: TCP
      port: 80
      targetPort: 8000
  type: LoadBalancer
"""
    return k8s_deployment.strip()

# 배포 파일 생성
print("📦 Docker 배포 파일 생성")
print("=" * 60)

# Dockerfile
dockerfile = create_dockerfile()
print("\n1. Dockerfile:")
print(dockerfile[:300] + "...")

# Docker Compose
docker_compose = create_docker_compose()
print("\n2. docker-compose.yml:")
print(docker_compose[:400] + "...")

# Kubernetes
k8s_config = create_kubernetes_deployment()
print("\n3. kubernetes-deployment.yaml:")
print(k8s_config[:400] + "...")

# 배포 명령어
print("\n🚀 배포 명령어:")
print("""
# Docker 빌드 및 실행
docker build -t langgraph-app .
docker run -p 8000:8000 --env-file .env langgraph-app

# Docker Compose
docker-compose up -d

# Kubernetes
kubectl apply -f kubernetes-deployment.yaml
kubectl get pods -l app=langgraph
kubectl get service langgraph-service
""")

## 5. Monitoring and Logging

In [ ]:
import logging
from datetime import datetime
import json
from typing import Any
import traceback

# 구조화된 로거
class StructuredLogger:
    """구조화된 로깅 시스템"""
    
    def __init__(self, name: str, level: str = "INFO"):
        self.logger = logging.getLogger(name)
        self.logger.setLevel(getattr(logging, level))
        
        # JSON 포맷터
        formatter = logging.Formatter(
            '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
        )
        
        # 핸들러 설정
        handler = logging.StreamHandler()
        handler.setFormatter(formatter)
        self.logger.addHandler(handler)
    
    def log_event(self, event_type: str, data: Dict[str, Any]):
        """이벤트 로깅"""
        log_entry = {
            "timestamp": datetime.now().isoformat(),
            "event_type": event_type,
            "data": data
        }
        self.logger.info(json.dumps(log_entry))
    
    def log_error(self, error: Exception, context: Dict[str, Any] = None):
        """에러 로깅"""
        error_entry = {
            "timestamp": datetime.now().isoformat(),
            "error_type": type(error).__name__,
            "error_message": str(error),
            "traceback": traceback.format_exc(),
            "context": context or {}
        }
        self.logger.error(json.dumps(error_entry))

# 메트릭 수집기
class MetricsCollector:
    """메트릭 수집 및 모니터링"""
    
    def __init__(self):
        self.metrics = {
            "counters": {},
            "gauges": {},
            "histograms": {}
        }
    
    def increment_counter(self, name: str, value: int = 1, labels: Dict = None):
        """카운터 증가"""
        key = self._create_key(name, labels)
        if key not in self.metrics["counters"]:
            self.metrics["counters"][key] = 0
        self.metrics["counters"][key] += value
    
    def set_gauge(self, name: str, value: float, labels: Dict = None):
        """게이지 설정"""
        key = self._create_key(name, labels)
        self.metrics["gauges"][key] = value
    
    def record_histogram(self, name: str, value: float, labels: Dict = None):
        """히스토그램 기록"""
        key = self._create_key(name, labels)
        if key not in self.metrics["histograms"]:
            self.metrics["histograms"][key] = []
        self.metrics["histograms"][key].append(value)
    
    def _create_key(self, name: str, labels: Dict = None) -> str:
        """메트릭 키 생성"""
        if labels:
            label_str = ",".join(f"{k}={v}" for k, v in sorted(labels.items()))
            return f"{name}{{{label_str}}}"
        return name
    
    def get_metrics(self) -> Dict:
        """모든 메트릭 반환"""
        return self.metrics
    
    def export_prometheus(self) -> str:
        """Prometheus 포맷으로 내보내기"""
        lines = []
        
        # Counters
        for key, value in self.metrics["counters"].items():
            lines.append(f"# TYPE {key.split('{')[0]} counter")
            lines.append(f"{key} {value}")
        
        # Gauges
        for key, value in self.metrics["gauges"].items():
            lines.append(f"# TYPE {key.split('{')[0]} gauge")
            lines.append(f"{key} {value}")
        
        # Histograms (simplified)
        for key, values in self.metrics["histograms"].items():
            if values:
                lines.append(f"# TYPE {key.split('{')[0]} histogram")
                lines.append(f"{key}_count {len(values)}")
                lines.append(f"{key}_sum {sum(values)}")
        
        return "\n".join(lines)

# 모니터링 데코레이터
def monitor_performance(logger: StructuredLogger, collector: MetricsCollector):
    """성능 모니터링 데코레이터"""
    def decorator(func):
        def wrapper(*args, **kwargs):
            start_time = time.time()
            
            try:
                # 함수 실행
                result = func(*args, **kwargs)
                
                # 성공 메트릭
                latency = time.time() - start_time
                collector.increment_counter(
                    "function_calls_total",
                    labels={"function": func.__name__, "status": "success"}
                )
                collector.record_histogram(
                    "function_duration_seconds",
                    latency,
                    labels={"function": func.__name__}
                )
                
                # 로깅
                logger.log_event("function_executed", {
                    "function": func.__name__,
                    "duration": latency,
                    "status": "success"
                })
                
                return result
                
            except Exception as e:
                # 실패 메트릭
                collector.increment_counter(
                    "function_calls_total",
                    labels={"function": func.__name__, "status": "error"}
                )
                
                # 에러 로깅
                logger.log_error(e, {"function": func.__name__})
                raise
        
        return wrapper
    return decorator

# 모니터링 시스템 테스트
logger = StructuredLogger("langgraph_app")
collector = MetricsCollector()

# 모니터링이 적용된 함수
@monitor_performance(logger, collector)
def process_graph_request(graph_name: str, data: Dict):
    """그래프 요청 처리"""
    import random
    time.sleep(random.uniform(0.1, 0.5))  # 시뮬레이션
    
    if random.random() < 0.1:  # 10% 실패율
        raise Exception("Random processing error")
    
    return {"result": "success", "graph": graph_name}

# 테스트 실행
print("📊 모니터링 시스템 테스트")
print("=" * 60)

for i in range(10):
    try:
        result = process_graph_request("chat", {"message": f"test_{i}"})
    except:
        pass

# 메트릭 출력
print("\n📈 수집된 메트릭:")
metrics = collector.get_metrics()
print(f"Counters: {metrics['counters']}")
print(f"Histograms: {len(metrics['histograms'])} entries")

print("\n📋 Prometheus 포맷:")
print(collector.export_prometheus()[:500] + "...")

## 실습 과제

1. 실제 LangGraph 애플리케이션을 Docker로 컨테이너화하기
2. CI/CD 파이프라인 구성하기 (GitHub Actions)
3. 프로덕션 환경 모니터링 대시보드 구축하기

In [ ]:
# 여기에 실습 코드를 작성하세요
